Eileen Zhang 2020/08/08

In [1]:
import numpy as np
import os
import time

#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# cifar10 查看

In [2]:
from tensorflow import keras
from keras.datasets import cifar10

In [3]:
(x_train, y_train_cats), (x_test, y_test_cats) = cifar10.load_data()

In [4]:
x_train = x_train/255.
x_test = x_test/255.

In [36]:
import matplotlib.pyplot as plt
%matplotlib inline

0 airplane 										
1 automobile 										
2 bird 										
3 cat 										
4 deer 										
5 dog 										
6 frog 										
7 horse 										
8 ship 										
9 truck

In [5]:
def mlp(units, dropout = 0.2,output_layer=True):
    layers = []
    if(output_layer):
        last = units[-1]
        units = units[:-1]
    for u in units:
        layers.append(keras.layers.BatchNormalization())
        layers.append(keras.layers.Dense(u, activation='relu'))
        layers.append(keras.layers.Dropout(dropout))
    if(output_layer):
        layers.append(keras.layers.Dense(last, activation=keras.activations.softmax))
    return layers

In [6]:
import tensorflow as tf

# cnn + relu(做对比使用 baseline)

In [97]:
layers = []
layers.append(keras.layers.Conv2D(32, 5, input_shape = [32, 32,3], padding="same", activation='relu'))
layers.append(keras.layers.Conv2D(128, 4, padding="same",activation='relu'))
layers.append(keras.layers.Conv2D(16, 2, padding="same",activation='relu'))
layers.append(keras.layers.MaxPool2D(2))
layers.append(keras.layers.Flatten())
layers += mlp([300,20,10])

In [98]:
model2 = keras.models.Sequential(layers)
model2.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

In [99]:
model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 32)        2432      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 128)       65664     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 16)        8208      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 4096)              0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 4096)              16384     
_________________________________________________________________
dense_15 (Dense)             (None, 300)              

In [100]:
%%time
model2.fit(x_train, y_train_cats, epochs= 10)

Epoch 1/10
1563/1563 [==============================] - 537s 343ms/step - loss: 1.6922 - accuracy: 0.4012
Epoch 2/10
1563/1563 [==============================] - 563s 360ms/step - loss: 1.3232 - accuracy: 0.5315
Epoch 3/10
1563/1563 [==============================] - 741s 474ms/step - loss: 1.1549 - accuracy: 0.5937
Epoch 4/10
1563/1563 [==============================] - 514s 329ms/step - loss: 1.0452 - accuracy: 0.6365
Epoch 5/10
1563/1563 [==============================] - 538s 344ms/step - loss: 0.9797 - accuracy: 0.6582
Epoch 6/10
1563/1563 [==============================] - 493s 315ms/step - loss: 0.9131 - accuracy: 0.6811
Epoch 7/10
1563/1563 [==============================] - 485s 310ms/step - loss: 0.8625 - accuracy: 0.6990
Epoch 8/10
1563/1563 [==============================] - 486s 311ms/step - loss: 0.8243 - accuracy: 0.7140
Epoch 9/10
1563/1563 [==============================] - 477s 305ms/step - loss: 0.7771 - accuracy: 0.7275
Epoch 10/10
1563/1563 [=======================

# SeparableConv2D

https://blog.csdn.net/shawroad88/article/details/95222082

![SeparableConv2D](../data/SeparableConv2D.png)

In [10]:
layers = []
layers.append(keras.layers.SeparableConv2D(32, 5, input_shape = [32, 32,3], padding="same", activation='relu'))
layers.append(keras.layers.SeparableConv2D(128, 4, padding="same",activation='relu'))
layers.append(keras.layers.SeparableConv2D(16, 2, padding="same",activation='relu'))
layers.append(keras.layers.MaxPool2D(2))
layers.append(keras.layers.Flatten())
layers += mlp([300,20,10])

In [11]:
model3 = keras.models.Sequential(layers)
model3.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

In [12]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 32, 32, 32)        203       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 32, 32, 128)       4736      
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 32, 32, 16)        2576      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 4096)              16384     
_________________________________________________________________
dense_3 (Dense)              (None, 300)              

In [13]:
%%time
model3.fit(x_train, y_train_cats, epochs= 10)

Epoch 1/10
1563/1563 [==============================] - 279s 179ms/step - loss: 1.6639 - accuracy: 0.3966
Epoch 2/10
1563/1563 [==============================] - 256s 164ms/step - loss: 1.3602 - accuracy: 0.5143
Epoch 3/10
1563/1563 [==============================] - 263s 168ms/step - loss: 1.2005 - accuracy: 0.5743
Epoch 4/10
1563/1563 [==============================] - 273s 175ms/step - loss: 1.0986 - accuracy: 0.6134
Epoch 5/10
1563/1563 [==============================] - 275s 176ms/step - loss: 1.0170 - accuracy: 0.6420
Epoch 6/10
1563/1563 [==============================] - 269s 172ms/step - loss: 0.9480 - accuracy: 0.6669
Epoch 7/10
1563/1563 [==============================] - 265s 169ms/step - loss: 0.9009 - accuracy: 0.6840
Epoch 8/10
1563/1563 [==============================] - 269s 172ms/step - loss: 0.8560 - accuracy: 0.6980
Epoch 9/10
1563/1563 [==============================] - 276s 176ms/step - loss: 0.8097 - accuracy: 0.7156
Epoch 10/10
1563/1563 [=======================

# DepthwiseConv2D

![DepthwiseConv2D](../data/DepthwiseConv2D.png)

**总结 :** 非常不错

In [15]:
layers = []
layers.append(keras.layers.DepthwiseConv2D(3, input_shape = [32, 32,3], padding="same", activation='relu'))
layers.append(keras.layers.Conv2D(32, 4, padding="same",activation='relu'))
layers.append(keras.layers.Conv2D(16, 2, padding="same",activation='relu'))
layers.append(keras.layers.MaxPool2D(2))
layers.append(keras.layers.Flatten())
layers += mlp([300,20,10])

In [16]:
model4 = keras.models.Sequential(layers)
model4.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

In [18]:
model4.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
depthwise_conv2d_1 (Depthwis (None, 32, 32, 3)         30        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        1568      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 16)        2064      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4096)              0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 4096)              16384     
_________________________________________________________________
dense_9 (Dense)              (None, 300)              

In [17]:
%%time
model4.fit(x_train, y_train_cats, epochs= 10)

Epoch 1/10
1563/1563 [==============================] - 131s 84ms/step - loss: 1.6193 - accuracy: 0.4267
Epoch 2/10
1563/1563 [==============================] - 128s 82ms/step - loss: 1.2041 - accuracy: 0.5755
Epoch 3/10
1563/1563 [==============================] - 126s 80ms/step - loss: 1.0609 - accuracy: 0.6296
Epoch 4/10
1563/1563 [==============================] - 126s 81ms/step - loss: 0.9703 - accuracy: 0.6617
Epoch 5/10
1563/1563 [==============================] - 124s 79ms/step - loss: 0.8954 - accuracy: 0.6857
Epoch 6/10
1563/1563 [==============================] - 126s 80ms/step - loss: 0.8508 - accuracy: 0.7018
Epoch 7/10
1563/1563 [==============================] - 126s 81ms/step - loss: 0.7842 - accuracy: 0.7267
Epoch 8/10
1563/1563 [==============================] - 126s 81ms/step - loss: 0.7406 - accuracy: 0.7391
Epoch 9/10
1563/1563 [==============================] - 125s 80ms/step - loss: 0.7005 - accuracy: 0.7531
Epoch 10/10
1563/1563 [==============================] 